# Explore Random Forest model with grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Read in & clean text

In [3]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('datasets/anti-bully-data.csv', header=0)

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['text_message'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['text_message'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['text_message'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,12011,12012,12013,12014,12015,12016,12017,12018,12019,12020
0,92,5.4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,99,5.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19,5.3,0.329286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,102,4.9,0.253523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100,5.0,0.155094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Build Grid-search

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label_bullying'], test_size=0.3)

In [18]:
 def train_RF(n_est, depth):
        rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
        rf_model = rf.fit(X_train, y_train)
        y_pred = rf_model.predict(X_test)
        precision, recall, fscore, support = score(y_test, y_pred,
                                                   pos_label=1,
                                                   average='binary')
        
        print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
            n_est, depth, round(precision, 3), round(recall, 3),
            round((y_pred==y_test).sum() / len(y_pred), 3)))

In [21]:
for n_est in [10, 50, 100, 300]:
    for depth in [10, 20, 30, 100, None]:
        train_RF(n_est, depth)

C:\Users\Jimmy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 10 / Depth: 10 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.721


C:\Users\Jimmy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 10 / Depth: 20 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.721
Est: 10 / Depth: 30 ---- Precision: 0.688 / Recall: 0.015 / Accuracy: 0.724
Est: 10 / Depth: 100 ---- Precision: 0.531 / Recall: 0.092 / Accuracy: 0.724
Est: 10 / Depth: None ---- Precision: 0.514 / Recall: 0.099 / Accuracy: 0.723


C:\Users\Jimmy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Est: 300 / Depth: 10 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.721


C:\Users\Jimmy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 300 / Depth: 20 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.721


C:\Users\Jimmy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 300 / Depth: 30 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.721
Est: 300 / Depth: 100 ---- Precision: 0.658 / Recall: 0.065 / Accuracy: 0.73
Est: 300 / Depth: None ---- Precision: 0.613 / Recall: 0.114 / Accuracy: 0.733
